In [1]:
import warnings
import time
import sys
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)

import winsound
frequency = 400  # Hz
duration = 700    # milliseconds

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
new_merchant_transactions = pd.read_csv('../Kaggle-data/new_merchant_transactions.csv')
                              #  ,parse_dates=['purchase_date'])

historical_transactions = pd.read_csv('../Kaggle-data/historical_transactions.csv')
                                      # ,parse_dates=['purchase_date'])
merchants = pd.read_csv('../Kaggle-data/merchants.csv')
train = pd.read_csv('../Kaggle-data/train.csv')
test = pd.read_csv('../Kaggle-data/test.csv')
winsound.Beep(frequency, duration)

In [4]:
# 問題が起きればここから回す
new_transactions = new_merchant_transactions.copy()
hist_transactions = historical_transactions.copy()
merchants_temp = merchants.copy()
train_temp = train.copy()
test_temp = test.copy()

In [5]:
# installmentsが999のものを削除
new_transactions = new_transactions[new_transactions['installments'] != 999]
hist_transactions = hist_transactions[hist_transactions['installments'] != 999]

In [4]:
merchants_temp.isnull().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [6]:
hist_transactions['purchase_amount_new'] = np.round(historical_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)

In [6]:
hist_transactions.head(5)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,29.00
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,9.17
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,17.65
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,7.69
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,16.00


In [7]:
# target < -30は除去
train_temp = train.query('target > -30')

In [8]:
hist_train = pd.merge(hist_transactions, train_temp, on='card_id', how='inner')
hist_train['target_before'] = 2 ** hist_train['target']

In [9]:
hist_test = pd.merge(hist_transactions, test, on='card_id', how='inner')

In [96]:
hist_train

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new,first_active_month,feature_1,feature_2,feature_3,target,target_before
0,N,C_ID_5037ff576e,322,N,1,B,278,M_ID_b61c7d1be0,-3,-0.595260,2017-09-07 20:57:19,3.0,11,37,100.92,2017-01,5,1,1,-2.352713,0.195778
1,Y,C_ID_5037ff576e,138,N,1,B,307,M_ID_fe69229f24,-4,1.189469,2017-08-14 15:46:08,1.0,15,19,1288.64,2017-01,5,1,1,-2.352713,0.195778
2,Y,C_ID_5037ff576e,138,N,1,B,705,M_ID_efc106141c,-9,-0.640069,2017-03-05 14:57:51,1.0,15,33,71.10,2017-01,5,1,1,-2.352713,0.195778
3,Y,C_ID_5037ff576e,226,N,1,B,307,M_ID_708022307c,-4,-0.652256,2017-08-15 10:45:28,1.0,16,19,62.99,2017-01,5,1,1,-2.352713,0.195778
4,Y,C_ID_5037ff576e,330,N,1,B,705,M_ID_393b4b8cec,-9,-0.674210,2017-03-26 20:00:31,3.0,17,33,48.38,2017-01,5,1,1,-2.352713,0.195778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17849899,Y,C_ID_2863d2fa95,-1,Y,1,B,360,M_ID_edd92b6720,-1,-0.632706,2017-01-20 08:52:04,NaN,-1,34,76.00,2016-05,1,2,0,-0.462466,0.725745
17849900,Y,C_ID_2863d2fa95,-1,Y,1,B,360,M_ID_edd92b6720,0,-0.632706,2017-02-20 04:40:50,NaN,-1,34,76.00,2016-05,1,2,0,-0.462466,0.725745
17849901,Y,C_ID_5c240d6e3c,3,N,0,A,278,M_ID_9cdcfe8673,0,-0.657740,2017-12-26 18:37:51,1.0,16,37,59.34,2017-10,2,2,0,0.719115,1.646172
17849902,Y,C_ID_5c240d6e3c,331,N,0,A,514,M_ID_1a75f94f92,-1,0.770620,2017-11-24 14:18:15,1.0,16,9,1009.90,2017-10,2,2,0,0.719115,1.646172


In [10]:
hist_train = hist_train[['card_id', 'merchant_id', 'purchase_date', 'month_lag', 'purchase_amount', 'purchase_amount_new', 'installments', 'target', 'target_before']]
# hist_train.query('month_lag == -1 or month_lag == -2').sort_values(by=['card_id', 'merchant_id', 'purchase_date']).head(15)

In [23]:
# # purchase_amount_newの変化率を出す関数
# def average_change_ratio(group):
#     group = group.sort_values('month_lag')
#     group['prev_purchase_amount'] = group['purchase_amount_new'].shift(-1)
#     group['monthly_change_rate'] = (group['purchase_amount_new'] - group['prev_purchase_amount']) / group['prev_purchase_amount']
#     return group

In [80]:
hist_train.head(10)

,card_id,merchant_id,purchase_date,month_lag,purchase_amount,purchase_amount_new,installments,target,target_before
0,C_ID_5037ff576e,M_ID_b61c7d1be0,2017-09-07 20:57:19,-3,-0.595260,100.92,1,-2.352713,0.195778
1,C_ID_5037ff576e,M_ID_fe69229f24,2017-08-14 15:46:08,-4,1.189469,1288.64,1,-2.352713,0.195778
2,C_ID_5037ff576e,M_ID_efc106141c,2017-03-05 14:57:51,-9,-0.640069,71.10,1,-2.352713,0.195778
3,C_ID_5037ff576e,M_ID_708022307c,2017-08-15 10:45:28,-4,-0.652256,62.99,1,-2.352713,0.195778
4,C_ID_5037ff576e,M_ID_393b4b8cec,2017-03-26 20:00:31,-9,-0.674210,48.38,1,-2.352713,0.195778
5,C_ID_5037ff576e,M_ID_fe69229f24,2017-03-09 18:32:05,-9,-0.671775,50.00,1,-2.352713,0.195778
6,C_ID_5037ff576e,M_ID_d70b0fd11b,2017-06-07 19:39:09,-6,-0.627477,79.48,1,-2.352713,0.195778
7,C_ID_5037ff576e,M_ID_16cf5cd707,2017-11-09 04:33:33,-1,-0.658296,58.97,1,-2.352713,0.195778
8,C_ID_5037ff576e,M_ID_6a5ddaaba7,2017-04-26 10:20:17,-8,-0.696569,33.50,1,-2.352713,0.195778
9,C_ID_5037ff576e,M_ID_46ab73a225,2017-10-19 21:54:37,-2,-0.521510,150.00,1,-2.352713,0.195778


In [11]:
hist_train_shift_1 = hist_train.groupby('card_id')['purchase_amount_new'].shift(1)
hist_train_shift_1 = hist_train_shift_1.reset_index().rename({'purchase_amount_new': 'purchase_amount_shifted'}, axis=1)
hist_train_combined = pd.concat([hist_train, hist_train_shift_1['purchase_amount_shifted']], axis=1)
# hist_train_combined
hist_train_combined['purchase_amount_change_ratio'] = (hist_train_combined['purchase_amount_new'] - hist_train_combined['purchase_amount_shifted']) / hist_train_combined['purchase_amount_shifted']
hist_train_combined = hist_train_combined[['card_id', 'merchant_id', 'month_lag', 'purchase_amount_change_ratio', 'installments', 'target_before', 'target']]
hist_train_combined

,card_id,merchant_id,month_lag,purchase_amount_change_ratio,installments,target_before,target
0,C_ID_5037ff576e,M_ID_b61c7d1be0,-3,NaN,1,0.195778,-2.352713
1,C_ID_5037ff576e,M_ID_fe69229f24,-4,11.768926,1,0.195778,-2.352713
2,C_ID_5037ff576e,M_ID_efc106141c,-9,-0.944826,1,0.195778,-2.352713
3,C_ID_5037ff576e,M_ID_708022307c,-4,-0.114065,1,0.195778,-2.352713
4,C_ID_5037ff576e,M_ID_393b4b8cec,-9,-0.231942,1,0.195778,-2.352713
...,...,...,...,...,...,...,...
17849796,C_ID_2863d2fa95,M_ID_edd92b6720,-1,NaN,1,0.725745,-0.462466
17849797,C_ID_2863d2fa95,M_ID_edd92b6720,0,0.000000,1,0.725745,-0.462466
17849798,C_ID_5c240d6e3c,M_ID_9cdcfe8673,0,NaN,0,1.646172,0.719115
17849799,C_ID_5c240d6e3c,M_ID_1a75f94f92,-1,16.018874,0,1.646172,0.719115


In [12]:
hist_test_shift_1 = hist_test.groupby('card_id')['purchase_amount_new'].shift(1)
hist_test_shift_1 = hist_test_shift_1.reset_index().rename({'purchase_amount_new': 'purchase_amount_shifted'}, axis=1)
hist_test_combined = pd.concat([hist_test, hist_test_shift_1['purchase_amount_shifted']], axis=1)
# hist_train_combined
hist_test_combined['purchase_amount_change_ratio'] = (hist_test_combined['purchase_amount_new'] - hist_test_combined['purchase_amount_shifted']) / hist_test_combined['purchase_amount_shifted']
hist_test_combined = hist_test_combined[['card_id', 'merchant_id', 'month_lag', 'purchase_amount_change_ratio', 'installments']]
hist_test_combined

,card_id,merchant_id,month_lag,purchase_amount_change_ratio,installments
0,C_ID_4e6213e9bc,M_ID_e020e9b302,-8,NaN,0
1,C_ID_4e6213e9bc,M_ID_86ec983688,-7,-0.683793,0
2,C_ID_4e6213e9bc,M_ID_979ed661fc,-6,0.924755,0
3,C_ID_4e6213e9bc,M_ID_e6d5ae8ea6,-5,-0.564306,0
4,C_ID_4e6213e9bc,M_ID_e020e9b302,-11,1.080624,0
...,...,...,...,...,...
11082265,C_ID_e43fce4842,M_ID_e50b4f17e7,-1,NaN,1
11082266,C_ID_e43fce4842,M_ID_00a6ca8a8a,-7,-0.969947,1
11082267,C_ID_e43fce4842,M_ID_3111c6df35,-6,329.971831,10
11082268,C_ID_112832329d,M_ID_cfa9a773ca,0,NaN,8


In [13]:
# 新たな特徴量として顧客ごとにchange_ratioの最大値、平均とmonth_lagの最小値の絶対値、平均、installmentsの最大値、平均値、最小値を作る for train
hist_train_max_ratio = hist_train_combined.groupby('card_id').purchase_amount_change_ratio.max().reset_index().rename({'purchase_amount_change_ratio': 'purchase_change_ratio_max'}, axis=1)
hist_train_mean_ratio = hist_train_combined.groupby('card_id').purchase_amount_change_ratio.mean().reset_index().rename({'purchase_amount_change_ratio': 'purchase_change_ratio_mean'}, axis=1)
hist_train_min_lag = hist_train_combined.groupby('card_id').month_lag.min().abs().reset_index().rename({'month_lag': 'month_lag_min'}, axis=1)
hist_train_mean_lag = hist_train_combined.groupby('card_id').month_lag.mean().reset_index().rename({'month_lag': 'month_lag_mean'}, axis=1)
hist_train_max_inst = hist_train_combined.groupby('card_id').installments.max().reset_index().rename({'installments': 'installments_max'}, axis=1)
hist_train_min_inst = hist_train_combined.groupby('card_id').installments.min().reset_index().rename({'installments': 'installments_min'}, axis=1)
hist_train_mean_inst = hist_train_combined.groupby('card_id').installments.mean().reset_index().rename({'installments': 'installments_min'}, axis=1)

hist_train_target_before = hist_train_combined.groupby('card_id').target_before.max().reset_index()
hist_train_target = hist_train_combined.groupby('card_id').target.max().reset_index()

In [14]:
# 新たな特徴量として顧客ごとにchange_ratioの最大値、平均とmonth_lagの最小値の絶対値、平均、installmentsの最大値、平均値、最小値を作る for test
hist_test_max_ratio = hist_test_combined.groupby('card_id').purchase_amount_change_ratio.max().reset_index().rename({'purchase_amount_change_ratio': 'purchase_change_ratio_max'}, axis=1)
hist_test_mean_ratio = hist_test_combined.groupby('card_id').purchase_amount_change_ratio.mean().reset_index().rename({'purchase_amount_change_ratio': 'purchase_change_ratio_mean'}, axis=1)
hist_test_min_lag = hist_test_combined.groupby('card_id').month_lag.min().abs().reset_index().rename({'month_lag': 'month_lag_min'}, axis=1)
hist_test_mean_lag = hist_test_combined.groupby('card_id').month_lag.mean().reset_index().rename({'month_lag': 'month_lag_mean'}, axis=1)
hist_test_max_inst = hist_test_combined.groupby('card_id').installments.max().reset_index().rename({'installments': 'installments_max'}, axis=1)
hist_test_min_inst = hist_test_combined.groupby('card_id').installments.min().reset_index().rename({'installments': 'installments_min'}, axis=1)
hist_test_mean_inst = hist_test_combined.groupby('card_id').installments.mean().reset_index().rename({'installments': 'installments_mean'}, axis=1)


In [15]:
# 上で作った特徴量をtrainデータと結合
df_train = pd.merge(train, hist_train_max_ratio, on='card_id', how='left')
df_train = pd.merge(df_train, hist_train_mean_ratio, on='card_id', how='left')
df_train = pd.merge(df_train, hist_train_min_lag, on='card_id', how='left')
df_train = pd.merge(df_train, hist_train_mean_lag, on='card_id', how='left')
df_train = pd.merge(df_train, hist_train_target_before, on='card_id', how='left')
df_train = pd.merge(df_train, hist_train_max_inst, on='card_id', how='left')
df_train = pd.merge(df_train, hist_train_min_inst, on='card_id', how='left')
df_train = pd.merge(df_train, hist_train_mean_inst, on='card_id', how='left')


In [16]:
# 上で作った特徴量をtestデータと結合
df_test = pd.merge(test, hist_test_max_ratio, on='card_id', how='left')
df_test = pd.merge(df_test, hist_test_mean_ratio, on='card_id', how='left')
df_test = pd.merge(df_test, hist_test_min_lag, on='card_id', how='left')
df_test = pd.merge(df_test, hist_test_mean_lag, on='card_id', how='left')
df_test = pd.merge(df_test, hist_test_max_inst, on='card_id', how='left')
df_test = pd.merge(df_test, hist_test_min_inst, on='card_id', how='left')
df_test = pd.merge(df_test, hist_test_mean_inst, on='card_id', how='left')

In [17]:
df_train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,purchase_change_ratio_max,purchase_change_ratio_mean,month_lag_min,month_lag_mean,target_before,installments_max,installments_min_x,installments_min_y
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,22.705000,1.466361,8.0,-3.911538,0.566331,1.0,0.0,0.015385
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,153.032857,3.368113,12.0,-5.031429,1.313042,10.0,-1.0,1.551429
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,5.481932,0.416662,13.0,-8.604651,1.611111,0.0,0.0,0.000000
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,71.954500,2.032829,5.0,-2.831169,1.103813,3.0,-1.0,1.090909
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,197.869402,6.707133,3.0,-1.285714,0.895181,12.0,1.0,1.368421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09,C_ID_963962de2c,3,2,1,-2.740821,24.839793,2.652121,5.0,-2.065217,0.149600,1.0,0.0,0.021739
201913,2015-10,C_ID_1314773c0b,3,1,1,0.312917,22.333333,1.853943,13.0,-3.872340,1.242217,1.0,0.0,0.021277
201914,2017-08,C_ID_7666735b3d,4,3,0,0.093494,214.719064,5.938508,6.0,-3.735632,1.066951,9.0,-1.0,1.241379
201915,2016-07,C_ID_73f5a0efd0,3,2,1,-4.676589,6.500000,0.560173,9.0,-3.346154,0.039103,0.0,0.0,0.000000


In [18]:
df_test

,first_active_month,card_id,feature_1,feature_2,feature_3,purchase_change_ratio_max,purchase_change_ratio_mean,month_lag_min,month_lag_mean,installments_max,installments_min,installments_mean
0,2017-04,C_ID_0ab67a22ab,3,3,1,19.708000,1.901517,8,-3.632353,12,1,2.073529
1,2017-01,C_ID_130fd0cbdd,2,3,0,30.300000,1.002160,13,-10.410256,4,1,1.064103
2,2017-08,C_ID_b709037bc5,5,1,1,3.939236,0.627736,6,-2.076923,10,-1,3.384615
3,2017-12,C_ID_d27d835a9f,2,1,0,10.623431,1.176950,2,-1.230769,6,-1,1.461538
4,2015-12,C_ID_2b5e3df5c2,5,1,1,56999.000000,550.092913,13,-6.227273,4,1,1.090909
...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10,C_ID_7a239d2eda,2,2,0,4.179898,0.364040,4,-1.767123,0,0,0.000000
123619,2017-09,C_ID_75ace375ae,3,1,1,8.355649,1.759106,5,-2.714286,0,0,0.000000
123620,2016-09,C_ID_21d56d950c,5,1,1,12.408693,1.353518,10,-3.837838,7,-1,1.594595
123621,2017-06,C_ID_6c46fc5a9d,2,1,0,16.937647,1.102436,4,-2.145161,5,1,1.935484


In [62]:
len(df_train)

201917

In [19]:
# データの保存
df_train.to_csv('../Kaggle-data/processed/kawai_processed_0621_train.csv',index=None)
df_test.to_csv('../Kaggle-data/processed/kawai_processed_0621_test.csv',index=None)

In [11]:
len(hist_train.query('merchant_id == "M_ID_fc7d7969c3"'))

109827

In [5]:
# 欠損値が1つでもあれば削除
new_transactions = new_transactions.dropna()
hist_transactions = hist_transactions.dropna()
merchants_temp = merchants_temp.dropna()

# installmentsが999のものを削除
new_transactions = new_transactions[new_transactions['installments'] != 999]
hist_transactions = hist_transactions[hist_transactions['installments'] != 999]

# installmentsが-1のものを削除
new_transactions = new_transactions[new_transactions['installments'] != -1]
hist_transactions = hist_transactions[hist_transactions['installments'] != -1]

# trainデータでターゲットが-30未満のものは削除
# train_temp = train_temp.query('target > -30')

In [7]:
merchants_temp.isnull().sum()

merchant_id                    0
merchant_group_id              0
merchant_category_id           0
subsector_id                   0
numerical_1                    0
numerical_2                    0
category_1                     0
most_recent_sales_range        0
most_recent_purchases_range    0
avg_sales_lag3                 0
avg_purchases_lag3             0
active_months_lag3             0
avg_sales_lag6                 0
avg_purchases_lag6             0
active_months_lag6             0
avg_sales_lag12                0
avg_purchases_lag12            0
active_months_lag12            0
category_4                     0
city_id                        0
state_id                       0
category_2                     0
dtype: int64

In [6]:
pd.set_option('display.max_columns', None)
merchants_temp.head(5)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
16,M_ID_c929bb59af,9514,2,20,-0.057471,-0.057471,N,E,E,69667.00,1.000000,3,69667.00,1.000000,3,69667.00,1.000000,3,N,-1,20,5.0
17,M_ID_dd3ae3de10,30534,278,37,-0.057471,-0.057471,N,E,E,5180.00,4.000000,3,9830.00,4.166667,6,15826.00,5.600000,10,Y,11,23,2.0


In [9]:
merchants_temp['most_recent_purchases_range'].unique()

array(['E', 'D', 'A', 'C', 'B'], dtype=object)

In [10]:
merchants_temp['category_2'].unique()

array([1., 5., 2., 3., 4.])

In [7]:
# Y/Nを1/0で置き換える
new_transactions['category_1'] = new_transactions['category_1'].map(lambda x: 1 if x=='Y' else 0)
hist_transactions['category_1'] = hist_transactions['category_1'].map(lambda x: 1 if x=='Y' else 0)
merchants_temp['category_1'] = merchants_temp['category_1'].map(lambda x: 1 if x=='Y' else 0)
merchants_temp['category_4'] = merchants_temp['category_4'].map(lambda x: 1 if x=='Y' else 0)

In [8]:
def encode_categorical_columns(df, columns):
    return pd.get_dummies(df, columns=columns)

new_transactions = encode_categorical_columns(new_transactions, ['category_2', 'category_3'])
hist_transactions = encode_categorical_columns(hist_transactions, ['category_2', 'category_3'])
merchants_temp = encode_categorical_columns(merchants_temp, ['category_2', 'most_recent_sales_range', 'most_recent_purchases_range'])


In [9]:
bool_columns = ['category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0', 'category_3_A', 'category_3_B', 'category_3_C']
new_transactions[bool_columns] = new_transactions[bool_columns].map(lambda x: 1 if x == True else 0)
hist_transactions[bool_columns] = hist_transactions[bool_columns].map(lambda x: 1 if x == True else 0)
new_transactions.head(5)

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,Y,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0
1,Y,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0
2,Y,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0
5,Y,C_ID_ef55cf8d4b,231,0,1,367,M_ID_8874615e00,2,-0.633007,2018-04-02 12:53:28,9,16,1,0,0,0,0,0,1,0
6,Y,C_ID_ef55cf8d4b,69,0,1,333,M_ID_6d061b5ddc,1,5.263697,2018-03-28 19:50:19,9,10,1,0,0,0,0,0,1,0


In [10]:
bool_columns_for_merchants = ['category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0',
                              'most_recent_sales_range_A', 'most_recent_sales_range_B', 'most_recent_sales_range_C', 'most_recent_sales_range_D', 'most_recent_sales_range_E',
                              'most_recent_purchases_range_A', 'most_recent_purchases_range_B', 'most_recent_purchases_range_C', 'most_recent_purchases_range_D', 'most_recent_purchases_range_E']

merchants_temp[bool_columns_for_merchants] = merchants_temp[bool_columns_for_merchants].map(lambda x: 1 if x == True else 0)
merchants_temp.head(5)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,most_recent_sales_range_A,most_recent_sales_range_B,most_recent_sales_range_C,most_recent_sales_range_D,most_recent_sales_range_E,most_recent_purchases_range_A,most_recent_purchases_range_B,most_recent_purchases_range_C,most_recent_purchases_range_D,most_recent_purchases_range_E
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,0,-0.40,9.666667,3,-2.25,18.666667,6,-2.32,13.916667,12,0,242,9,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,0,-0.72,1.750000,3,-0.74,1.291667,6,-0.57,1.687500,12,0,22,16,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,0,-82.13,260.000000,2,-82.13,260.000000,2,-82.13,260.000000,2,0,-1,5,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
16,M_ID_c929bb59af,9514,2,20,-0.057471,-0.057471,0,69667.00,1.000000,3,69667.00,1.000000,3,69667.00,1.000000,3,0,-1,20,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
17,M_ID_dd3ae3de10,30534,278,37,-0.057471,-0.057471,0,5180.00,4.000000,3,9830.00,4.166667,6,15826.00,5.600000,10,1,11,23,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1


In [11]:
# purchase_monthを取得
new_transactions['purchase_month'] = pd.to_datetime(new_transactions['purchase_date']).dt.to_period('M')
hist_transactions['purchase_month'] = pd.to_datetime(hist_transactions['purchase_date']).dt.to_period('M')
new_transactions.head(5)

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,purchase_month
0,Y,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0,2018-03
1,Y,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0,2018-03
2,Y,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0,2018-04
5,Y,C_ID_ef55cf8d4b,231,0,1,367,M_ID_8874615e00,2,-0.633007,2018-04-02 12:53:28,9,16,1,0,0,0,0,0,1,0,2018-04
6,Y,C_ID_ef55cf8d4b,69,0,1,333,M_ID_6d061b5ddc,1,5.263697,2018-03-28 19:50:19,9,10,1,0,0,0,0,0,1,0,2018-03


In [12]:
new_transactions['authorized_flag'] = new_transactions['authorized_flag'].map(lambda x: 1 if x == 'Y' else 0)
hist_transactions['authorized_flag'] = hist_transactions['authorized_flag'].map(lambda x: 1 if x == 'Y' else 0)

In [13]:
# 顧客ごとに量的変数のみ平均をとって残りのカテゴリ変数と結合
new_for_quantity = new_transactions.groupby('card_id')[['month_lag', 'installments', 'purchase_amount']].mean().reset_index()
new_for_category = new_transactions.groupby('card_id')[['category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0',
                                                        'category_3_A', 'category_3_B', 'category_3_C']].sum()
new_for_train = pd.merge(new_for_quantity, new_for_category, on='card_id', how='inner')

new_for_train.head(5)

,card_id,month_lag,installments,purchase_amount,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,C_ID_00007093c1,2.000000,1.000000,-0.664262,1,0,1,0,0,0,2,0
1,C_ID_0001238066,1.363636,1.590909,-0.577139,19,0,0,0,3,0,19,3
2,C_ID_0001506ef0,1.000000,0.000000,-0.732001,0,0,1,0,0,1,0,0
3,C_ID_0001793786,1.275862,0.000000,0.031598,15,8,5,0,1,29,0,0
4,C_ID_000183fdda,1.200000,1.700000,-0.648310,0,0,10,0,0,0,6,4


In [14]:
# 顧客ごとに量的変数のみ平均をとって残りのカテゴリ変数と結合
hist_for_quantity = hist_transactions.groupby('card_id')[['month_lag', 'installments', 'purchase_amount']].mean().reset_index()
hist_for_category = hist_transactions.groupby('card_id')[['category_2_1.0', 'category_2_2.0', 'category_2_3.0', 'category_2_4.0', 'category_2_5.0',
                                                        'category_3_A', 'category_3_B', 'category_3_C']].sum()
hist_for_train = pd.merge(hist_for_quantity, hist_for_category, on='card_id', how='inner')

hist_for_train.head(5)

,card_id,month_lag,installments,purchase_amount,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,C_ID_00007093c1,-6.074380,1.355372,-0.469471,0,0,120,0,1,0,97,24
1,C_ID_0001238066,-1.805310,1.654867,-0.609094,94,0,0,0,19,0,83,30
2,C_ID_0001506ef0,-4.833333,0.015152,-0.524271,2,0,64,0,0,65,1,0
3,C_ID_0001793786,-3.725490,0.009804,-0.129602,11,76,15,0,0,101,1,0
4,C_ID_000183fdda,-2.485294,1.779412,-0.515604,7,1,127,0,1,0,102,34


In [15]:
new_hist = pd.merge(hist_for_train, new_for_train, on='card_id', how='left')
new_hist['month_lag_mean'] = new_hist[['month_lag_x', 'month_lag_y']].mean(axis=1)
# new_hist['purchase_month'] = new_hist.groupby('card_id')[['purchase_month_x', 'purchase_month_y']].mean()
new_hist['installments_mean'] = new_hist[['installments_x', 'installments_y']].mean(axis=1)
new_hist['purchase_amount_mean'] = new_hist[['purchase_amount_x', 'purchase_amount_y']].mean(axis=1)
new_hist['category_2_1.0_count'] = new_hist[['category_2_1.0_x', 'category_2_1.0_y']].sum(axis=1)
new_hist['category_2_2.0_count'] = new_hist[['category_2_2.0_x', 'category_2_2.0_y']].sum(axis=1)
new_hist['category_2_3.0_count'] = new_hist[['category_2_3.0_x', 'category_2_3.0_y']].sum(axis=1)
new_hist['category_2_4.0_count'] = new_hist[['category_2_4.0_x', 'category_2_4.0_y']].sum(axis=1)
new_hist['category_2_5.0_count'] = new_hist[['category_2_5.0_x', 'category_2_5.0_y']].sum(axis=1)
new_hist['category_3_A_count'] = new_hist[['category_3_A_x', 'category_3_A_y']].sum(axis=1)
new_hist['category_3_B_count'] = new_hist[['category_3_B_x', 'category_3_B_y']].sum(axis=1)
new_hist['category_3_C_count'] = new_hist[['category_3_C_x', 'category_3_C_y']].sum(axis=1)
new_hist = new_hist[['card_id', 'month_lag_mean', 'installments_mean', 'purchase_amount_mean',
                     'category_2_1.0_count', 'category_2_2.0_count', 'category_2_3.0_count', 'category_2_4.0_count', 'category_2_5.0_count',
                     'category_3_A_count', 'category_3_B_count', 'category_3_C_count']]
new_hist.head(5)
new_hist.isnull().sum()

card_id                 0
month_lag_mean          0
installments_mean       0
purchase_amount_mean    0
category_2_1.0_count    0
category_2_2.0_count    0
category_2_3.0_count    0
category_2_4.0_count    0
category_2_5.0_count    0
category_3_A_count      0
category_3_B_count      0
category_3_C_count      0
dtype: int64

In [16]:
df_train = pd.merge(train, new_hist, on='card_id', how='left')
df_test = pd.merge(test, new_hist, on='card_id', how='left')

# どちらもcard_idに重複がないことを確認

In [79]:
df_train['purchase_amount_mean_new'] = 2 ** df_train['purchase_amount_mean']
df_test['purchase_amount_mean_new'] = 2 ** df_test['purchase_amount_mean']

df_train['target_before'] = 2 ** df_train['target']

In [81]:
df_train.describe()

C:\Users\toshiki.kawai.sx\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,feature_1,feature_2,feature_3,target,month_lag_mean,installments_mean,purchase_amount_mean,category_2_1.0_count,category_2_2.0_count,category_2_3.0_count,category_2_4.0_count,category_2_5.0_count,category_3_A_count,category_3_B_count,category_3_C_count,purchase_amount_mean_new,target_before
count,201917.000000,201917.000000,201917.000000,201917.000000,198253.000000,198253.000000,198253.000000,198253.000000,198253.000000,198253.000000,198253.000000,198253.000000,198253.000000,198253.000000,198253.000000,1.982530e+05,2.019170e+05
mean,3.105311,1.745410,0.565569,-0.393636,-1.636832,0.822988,1.090926,50.203719,3.382526,12.896380,8.643607,12.210262,49.593076,33.101325,4.642094,inf,6.820184e+00
std,1.186160,0.751362,0.495683,3.850500,1.805803,1.012548,563.642947,85.044580,24.119548,44.222651,40.331592,45.218510,95.050867,66.715062,8.792052,NaN,7.775483e+02
min,1.000000,1.000000,0.000000,-33.219281,-13.000000,0.000000,-0.746758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.959414e-01,1.000000e-10
25%,2.000000,1.000000,0.000000,-0.883110,-2.368421,0.000000,-0.680147,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.241017e-01,5.421973e-01
50%,3.000000,2.000000,1.000000,-0.023437,-1.256944,1.000000,-0.628269,18.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,0.000000,6.469520e-01,9.838860e-01
75%,4.000000,2.000000,1.000000,0.765453,-0.347222,1.294737,-0.521781,61.000000,0.000000,2.000000,0.000000,1.000000,62.000000,37.000000,6.000000,6.965113e-01,1.699904e+00
max,5.000000,3.000000,1.000000,17.965068,1.000000,12.000000,250441.166245,2164.000000,2554.000000,1218.000000,1388.000000,1224.000000,2890.000000,1385.000000,374.000000,inf,2.558730e+05


In [93]:
df_train.head(5)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,month_lag_mean,installments_mean,purchase_amount_mean,category_2_1.0_count,category_2_2.0_count,category_2_3.0_count,category_2_4.0_count,category_2_5.0_count,category_3_A_count,category_3_B_count,category_3_C_count,purchase_amount_mean_new,target_before
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,-1.252996,0.007874,-0.606428,274.0,0.0,0.0,0.0,3.0,273.0,4.0,0.0,0.656821,0.566331
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,-1.626972,1.250789,-0.674180,323.0,0.0,0.0,0.0,0.0,0.0,264.0,59.0,0.626688,1.313042
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,-3.302326,0.000000,-0.689319,4.0,0.0,0.0,0.0,40.0,44.0,0.0,0.0,0.620147,1.611111
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,-0.731746,1.007937,-0.693216,12.0,0.0,0.0,56.0,0.0,0.0,67.0,1.0,0.618473,1.103813
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,0.151127,1.000000,-0.600981,12.0,0.0,7.0,130.0,1.0,0.0,150.0,0.0,0.659306,0.895181


In [27]:
# データの保存
df_train.to_csv('../Kaggle-data/processed/kawai_processed_0619_train.csv',index=None)
df_test.to_csv('../Kaggle-data/processed/kawai_processed_0619_test.csv',index=None)

In [22]:
len(train)

201917

In [23]:
len(test)

123623

In [28]:
new_hist.head(5)

,card_id,month_lag_mean,installments_mean,purchase_amount_mean,category_2_1.0_count,category_2_2.0_count,category_2_3.0_count,category_2_4.0_count,category_2_5.0_count,category_3_A_count,category_3_B_count,category_3_C_count
0,C_ID_00007093c1,-2.037190,1.177686,-0.566867,1.0,0.0,121.0,0.0,1.0,0.0,99.0,24.0
1,C_ID_0001238066,-0.220837,1.622888,-0.593116,113.0,0.0,0.0,0.0,22.0,0.0,102.0,33.0
2,C_ID_0001506ef0,-1.916667,0.007576,-0.628136,2.0,0.0,65.0,0.0,0.0,66.0,1.0,0.0
3,C_ID_0001793786,-1.224814,0.004902,-0.049002,26.0,84.0,20.0,0.0,1.0,130.0,1.0,0.0
4,C_ID_000183fdda,-0.642647,1.739706,-0.581957,7.0,1.0,137.0,0.0,1.0,0.0,108.0,38.0


In [18]:
df_train.query('target < -30').describe()

,feature_1,feature_2,feature_3,target,month_lag_mean,installments_mean,purchase_amount_mean,category_2_1.0_count,category_2_2.0_count,category_2_3.0_count,category_2_4.0_count,category_2_5.0_count,category_3_A_count,category_3_B_count,category_3_C_count
count,2207.000000,2207.000000,2207.000000,2.207000e+03,2139.000000,2139.000000,2139.000000,2139.000000,2139.000000,2139.000000,2139.000000,2139.000000,2139.000000,2139.000000,2139.000000
mean,3.224286,1.782510,0.591300,-3.321928e+01,-2.459353,1.088606,1.086114,40.143525,3.289388,9.796634,8.561945,7.683497,25.755493,37.714820,6.004675
std,1.207783,0.810069,0.491705,7.107038e-15,2.058132,1.099055,69.808658,59.150863,22.681550,33.395751,38.589227,26.637457,59.313626,58.908818,8.988610
min,1.000000,1.000000,0.000000,-3.321928e+01,-12.200000,0.000000,-0.739565,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,0.000000,-3.321928e+01,-3.404509,0.010870,-0.677229,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,3.000000,2.000000,1.000000,-3.321928e+01,-1.865385,1.075000,-0.621195,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,2.000000
75%,4.000000,2.000000,1.000000,-3.321928e+01,-0.950000,1.453624,-0.505251,55.000000,0.000000,1.000000,0.000000,1.000000,29.000000,52.000000,9.000000
max,5.000000,3.000000,1.000000,-3.321928e+01,0.666667,9.666667,3226.854906,653.000000,636.000000,522.000000,732.000000,348.000000,760.000000,657.000000,90.000000


In [27]:
df_train.query('target > -30').describe()

,feature_1,feature_2,feature_3,target,month_lag_mean,installments_mean,purchase_amount_mean,category_2_1.0_count,category_2_2.0_count,category_2_3.0_count,category_2_4.0_count,category_2_5.0_count,category_3_A_count,category_3_B_count,category_3_C_count
count,199710.000000,199710.000000,199710.000000,199710.000000,196114.000000,196114.000000,196114.000000,196114.000000,196114.000000,196114.000000,196114.000000,196114.000000,196114.000000,196114.000000,196114.000000
mean,3.103996,1.745000,0.565285,-0.030879,-1.627861,0.820091,1.090979,50.313445,3.383542,12.930189,8.644498,12.259635,49.853070,33.051006,4.627232
std,1.185855,0.750679,0.495721,1.717793,1.800792,1.011183,566.661556,85.277233,24.134812,44.325032,40.350274,45.376799,95.334092,66.793551,8.788743
min,1.000000,1.000000,0.000000,-17.608147,-13.000000,0.000000,-0.746758,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,0.000000,-0.844513,-2.361111,0.000000,-0.680183,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.000000,2.000000,1.000000,-0.007783,-1.250000,1.000000,-0.628343,18.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.000000,0.000000
75%,4.000000,2.000000,1.000000,0.777331,-0.340541,1.292919,-0.521958,61.000000,0.000000,2.000000,0.000000,1.000000,62.000000,37.000000,6.000000
max,5.000000,3.000000,1.000000,17.965068,1.000000,12.000000,250441.166245,2164.000000,2554.000000,1218.000000,1388.000000,1224.000000,2890.000000,1385.000000,374.000000


In [25]:
df_train.query('target < -30 & purchase_amount_mean > 0')['purchase_amount_mean'].nunique()

126

In [60]:
df_train.query('target < -30 & purchase_amount_mean > 0').target.unique()

array([-33.21928095])

In [70]:
2 ** (-33.21928095)

9.999999992192566e-11

In [75]:
# 異常値は-33.21928095
epsilon = 1e-10
np.log2(epsilon)

np.float64(-33.219280948873624)